# Are accesses being made through mobile devices

In [1]:
import pandas as pd

## Loading active reports

In [2]:
active_reports = pd.read_csv("datasets/active_reports.csv", low_memory=False)

In [3]:
active_reports.shape

(38285, 51)

## Loading lightning performance

In [4]:
lightning_performance = pd.read_csv("data/Salesforce/Salesforce/ELF/LightningPerformance/2022-06-04_LightningPerformance.csv"
                                    , low_memory=False)

In [5]:
list(lightning_performance.columns)

['EVENT_TYPE',
 'TIMESTAMP',
 'REQUEST_ID',
 'ORGANIZATION_ID',
 'USER_ID',
 'CLIENT_ID',
 'SESSION_KEY',
 'LOGIN_KEY',
 'USER_TYPE',
 'APP_NAME',
 'DEVICE_PLATFORM',
 'SDK_APP_VERSION',
 'OS_NAME',
 'OS_VERSION',
 'USER_AGENT',
 'BROWSER_NAME',
 'BROWSER_VERSION',
 'SDK_VERSION',
 'DEVICE_MODEL',
 'DEVICE_ID',
 'SDK_APP_TYPE',
 'CLIENT_GEO',
 'CONNECTION_TYPE',
 'UI_EVENT_ID',
 'UI_EVENT_TYPE',
 'UI_EVENT_SOURCE',
 'UI_EVENT_TIMESTAMP',
 'PAGE_START_TIME',
 'DURATION',
 'DEVICE_SESSION_ID',
 'TIMESTAMP_DERIVED',
 'USER_ID_DERIVED',
 'CLIENT_IP']

## Merge active reports with lightning performance

In [6]:
common_columns = ['USER_ID_DERIVED', 'SESSION_KEY', 'LOGIN_KEY', 'ORGANIZATION_ID', 'CLIENT_IP']

In [7]:
lightning_performance.drop(['EVENT_TYPE','TIMESTAMP','REQUEST_ID','USER_ID','USER_TYPE','TIMESTAMP_DERIVED']
                           , inplace=True, axis=1)

In [8]:
ltng_full_reports_performance = pd.merge(left=active_reports, right=lightning_performance, on=common_columns)

In [9]:
ltng_full_reports_performance.shape

(7176291, 73)

In [10]:
list(ltng_full_reports_performance.columns)

['CreatedById',
 'CreatedDate',
 'Description',
 'DeveloperName',
 'FolderName',
 'Format',
 'Id',
 'IsDeleted',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastRunDate',
 'LastViewedDate',
 'Name',
 'NamespacePrefix',
 'OwnerId',
 'ReportTypeApiName',
 'SystemModstamp',
 'EVENT_TYPE',
 'TIMESTAMP',
 'REQUEST_ID',
 'ORGANIZATION_ID',
 'USER_ID',
 'RUN_TIME',
 'CPU_TIME',
 'URI',
 'SESSION_KEY',
 'LOGIN_KEY',
 'USER_TYPE',
 'REQUEST_STATUS',
 'DB_TOTAL_TIME',
 'ENTITY_NAME',
 'DISPLAY_TYPE',
 'RENDERING_TYPE',
 'REPORT_ID',
 'ROW_COUNT',
 'NUMBER_EXCEPTION_FILTERS',
 'NUMBER_COLUMNS',
 'AVERAGE_ROW_SIZE',
 'SORT',
 'DB_BLOCKS',
 'DB_CPU_TIME',
 'NUMBER_BUCKETS',
 'TIMESTAMP_DERIVED',
 'USER_ID_DERIVED',
 'CLIENT_IP',
 'URI_ID_DERIVED',
 'REPORT_ID_DERIVED',
 'ORIGIN',
 'IsActiveSinceCreation',
 'IsActiveSinceLastModification',
 'CLIENT_ID',
 'APP_NAME',
 'DEVICE_PLATFORM',
 'SDK_APP_VERSION',
 'OS_NAME',
 'OS_VERSION',
 'USER_AGENT',
 'BROWSER_NAME',
 'BROWSER_VER

In [11]:
ltng_full_reports_performance['DEVICE_MODEL'].unique()

array([nan], dtype=object)

## Problematic reports

In [12]:
threshold = 120000

In [13]:
ltng_full_reports_performance  = ltng_full_reports_performance[ltng_full_reports_performance.DURATION >= threshold]

In [14]:
ltng_full_reports_performance.shape

(86196, 73)

### Analyzing ***Device_Model***

In [15]:
interest_columns = ['DEVICE_ID', 'DEVICE_MODEL']

In [16]:
ltng_full_reports_performance_simplified = ltng_full_reports_performance[interest_columns]

In [17]:
ltng_full_reports_performance_simplified.shape

(86196, 2)

In [18]:
ltng_full_reports_performance_simplified['DEVICE_MODEL'].unique()

array([nan], dtype=object)